In [21]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform, randint
import pandas as pd
import numpy as np

df = pd.read_csv('data_preprocessed_V3.csv')

In [23]:
# Define features and target
X = df.drop('price', axis=1)
y = df['price']

# Split into train_val and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split train_val into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# Define the base XGBoost model
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    random_state=42
)

In [24]:
# Expanded parameter distribution for tuning
param_dist = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(3, 10),
    'min_child_weight': randint(1, 10),
    'gamma': uniform(0, 0.5),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'learning_rate': uniform(0.01, 0.19),
    'reg_alpha': uniform(0, 10),
    'reg_lambda': uniform(0, 10)
}

In [25]:
# Perform random search with cross-validation
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=100,
    scoring='neg_mean_squared_error',
    cv=KFold(n_splits=5, shuffle=True, random_state=42),
    verbose=1,
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=Non...
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001C1F073DD60>,
                                        'reg_alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001C1EC5BCAC0>,
                                        'reg_lambda': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001C1EC5E0490>,
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001C1EC5E0EE0>},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [27]:
# Get the best parameters
best_params = random_search.best_params_
print("Best parameters:", best_params)

# Train the final model with early stopping
final_model = xgb.XGBRegressor(
    **best_params,
    objective='reg:squarederror',
    early_stopping_rounds=20,
    random_state=42
)

final_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)

Best parameters: {'colsample_bytree': 0.797210263108722, 'gamma': 0.04164220559762982, 'learning_rate': 0.027423787979111408, 'max_depth': 6, 'min_child_weight': 9, 'n_estimators': 990, 'reg_alpha': 2.1272789502422897, 'reg_lambda': 9.461945428816902, 'subsample': 0.9125184201884811}


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.797210263108722, device=None,
             early_stopping_rounds=20, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.04164220559762982,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.027423787979111408,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=9, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=990, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [28]:
# Predict on the test set
y_pred = final_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')

MSE: 3099439129.0955873
RMSE: 55672.606631049595


In [32]:
from sklearn.metrics import r2_score
# Evaluate on training and test sets
y_train_pred = final_model.predict(X_train)
y_test_pred = final_model.predict(X_test)
print("Training R²:", r2_score(y_train, y_train_pred))
print("Test R²:", r2_score(y_test, y_test_pred))

Training R²: 0.9038266467124034
Test R²: 0.7871792290966197
